In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-62df470c-2f68-4dd5-56cb-4ae5e6f10a82)


In [ ]:
!pip install wget
!wget "https://scholar.cu.edu.eg/Dataset_BUSI.zip"

In [ ]:
!unzip -qq "/content/Dataset_BUSI.zip"

In [2]:
# ==== Install Dependencies
!pip install -q efficientnet-pytorch
!pip install -q albumentations
!pip install -U -q pytorch-fanatics 
!pip install -q pytorch_ranger

In [3]:
# ==== Import Libraries

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import seaborn as sns
import random
import os


import albumentations as aug
from albumentations.pytorch.transforms import ToTensor
import matplotlib.pyplot as plt


from tqdm import tqdm

from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils.data import Dataset,DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_fanatics.dataloader import Cloader
from pytorch_fanatics.utils import EarlyStop ,LRFinder 
from pytorch_fanatics.trainer import Trainer
from pytorch_fanatics.logger import Logger

import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from pytorch_ranger import Ranger

from efficientnet_pytorch import EfficientNet
from pathlib import Path

from torchvision import transforms

In [4]:
import os

asps = []
for root, dirs, files in os.walk('/content/Dataset_BUSI_with_GT/benign/'):
    # print(root)
    # print(dirs)
    # print(files)
    for file in files:
        if file.endswith('.png'):
            asps.append(file)

In [5]:
image_id=[]
labels=[]
for x in range(len(asps)):
  asps[x]=os.path.join('/content/Dataset_BUSI_with_GT/benign',asps[x])
  image_id.append(asps[x])
  labels.append(0)

In [6]:
import os

asps = []
for root, dirs, files in os.walk('/content/Dataset_BUSI_with_GT/malignant/'):
    # print(root)
    # print(dirs)
    # print(files)
    for file in files:
        if file.endswith('.png'):
            asps.append(file)

In [7]:
# image_id=[]
for x in range(len(asps)):
  asps[x]=os.path.join('/content/Dataset_BUSI_with_GT/malignant/',asps[x])
  image_id.append(asps[x])
  labels.append(1)

In [8]:
import os
asps = []
for root, dirs, files in os.walk('/content/Dataset_BUSI_with_GT/normal/'):
    # print(root)
    # print(dirs)
    # print(files)
    for file in files:
        if file.endswith('.png'):
            asps.append(file)

In [9]:
# image_id=[]
for x in range(len(asps)):
  asps[x]=os.path.join('/content/Dataset_BUSI_with_GT/normal/',asps[x])
  image_id.append(asps[x])
  labels.append(2)

In [10]:
d={'image_id':image_id,'labels':labels}
df=pd.DataFrame(d)

In [11]:
df=df.sample(frac=1)

In [ ]:
df=df.reset_index(drop=True)
df

In [ ]:
map={'Benign':0,'Malignant':1,'Normal':2}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
from sklearn.model_selection import train_test_split as tts

In [14]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

# Data Exploration

In [ ]:
!mkdir "/content/drive/MyDrive/HackNagpur/"

In [16]:
save_root = "/content/drive/MyDrive/HackNagpur/"

In [17]:
# training_data_path = "/content/ISIC_2019_Training_Input/ISIC_2019_Training_Input"

X_train , X_val ,Y_train , Y_val = tts(df, df.labels.values, test_size=0.25
                                       ,random_state=42,stratify=df.labels.values)
X_train       = X_train.reset_index(drop=True)
X_val         = X_val.reset_index(drop=True)

In [31]:
# ===== Augmentations

mean       = (0.485, 0.456, 0.406)
std        = (0.229, 0.224, 0.225)
image_size=224
train_tfms = aug.Compose([
            aug.Resize(224,224),
            aug.RandomSizedCrop(min_max_height=(64,224),height=224,width=224,p=0.5),
            aug.HorizontalFlip(p=0.5),
            aug.RandomBrightnessContrast(0.1,0.1),
            #aug.HueSaturationValue(10,10,10),
            aug.RGBShift(),
            aug.RandomContrast(limit=0.2),
            aug.RandomGamma(),
            #aug.ShiftScaleRotate(rotate_limit=(-45,45)),
            aug.GaussNoise(p=0.35),
            aug.IAASharpen(),
            aug.ToGray(p=0.35),
            aug.CLAHE(clip_limit=4.0, p=0.7),
            #aug.RandomSizedCrop(min_max_height=(64,256),height=256,width=256,p=0.5),
            aug.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
            aug.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
            #aug.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
            aug.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
            ])

test_tfms  = aug.Compose([
            aug.Resize(224,224),
            aug.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
            ])

In [ ]:
# training_data_path='/content/train'

In [32]:
import numpy as np
from PIL import Image
from PIL import ImageFile   #ImageFile contains support for PIL to open and save images
ImageFile.LOAD_TRUNCATED_IMAGES=True  #If image is truncated(or corrupt) then also load it..
import torch

class Cloader:
    def __init__(self,image_path,targets,resize=None,transforms=None):
        self.image_path=image_path
        self.targets=targets
        self.resize=resize
        self.transforms=transforms
        
    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self,idx):
        image = Image.open(self.image_path[idx]).convert("RGB")
        targets = self.targets[idx]
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )
        image = np.array(image)
        if self.transforms is not None:
            augmented = self.transforms(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }

In [33]:
train_images     = X_train.image_id.values.tolist()
# train_images     = [os.path.join(training_data_path, i+".png") for i in train_images]

test_images      = X_val.image_id.values.tolist()
# test_images      = [os.path.join(training_data_path, i+".png") for i in test_images]

train_dataset    = Cloader(train_images,X_train.labels.values,(224,224),train_tfms)
#train_dataset    = CutMix(train_dataset, num_class=8, beta=1.0, prob=0.5, num_mix=2)
test_dataset     = Cloader(test_images,X_val.labels.values,(224,224),test_tfms)

train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=0)
val_dataloader   = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=0)

device           = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [36]:
# ===== Define model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.base_model=EfficientNet.from_pretrained('efficientnet-b0',num_classes=3)
    def forward(self, image, targets):
        batch_size, _, _, _ = image.shape
        out = self.base_model(image)
        targets = torch.tensor(targets,dtype=torch.int64)
        loss = nn.CrossEntropyLoss()(out.view(batch_size,3), targets)
        return out, loss

model = Net()
model.to(device);

Loaded pretrained weights for efficientnet-b0


In [22]:
def softmax(array):
    return np.exp(array)/np.sum(np.exp(array),axis=1).reshape(-1,1)

In [39]:
optimizer = Ranger(model.parameters(),lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer,factor=0.8, mode="min", patience=2)

trainer   = Trainer(model=model,optimizer=optimizer,device=device,val_scheduler=scheduler)
logger    = Logger()

es        = EarlyStop(patience=5,mode="min") # mode = min to minimise loss

In [40]:
epochs = 30

for epoch in range(epochs):
    logger.write(f"+ ===== Epoch {epoch+1}/{epochs} ===== +")
    train_loss              = trainer.train(train_dataloader)
    y_true,y_pred ,val_loss = trainer.evaluate(val_dataloader)
    y_pred                  = softmax(y_pred)
    accuracy                = accuracy_score(y_true,np.argmax(y_pred,axis=1))
    es(val_loss,model,model_path ="/content/drive/MyDrive/HackNagpur/best.pth")
    logger.write(f"train_loss {train_loss} val_loss {val_loss} ")
    logger.write(f"val accuracy_score {accuracy} ")
    logger.write(" ")
    if es.early_stop:
        break

+ ===== Epoch 1/30 ===== +



Metric Validation score improved (inf --> 1.0334825011400077). Saving model!
train_loss 1.0860318686511063 val_loss 1.0334825011400077 
val accuracy_score 0.5240506329113924 
 
+ ===== Epoch 2/30 ===== +



Metric Validation score improved (1.0334825011400077 --> 1.008012097615462). Saving model!
train_loss 1.067051690977973 val_loss 1.008012097615462 
val accuracy_score 0.5088607594936709 
 
+ ===== Epoch 3/30 ===== +



Metric Validation score improved (1.008012097615462 --> 0.9630793883250308). Saving model!
train_loss 1.0282192391318243 val_loss 0.9630793883250308 
val accuracy_score 0.6886075949367089 
 
+ ===== Epoch 4/30 ===== +



Metric Validation score improved (0.9630793883250308 --> 0.8941195973983178). Saving model!
train_loss 0.991892516613006 val_loss 0.8941195973983178 
val accuracy_score 0.7316455696202532 
 
+ ===== Epoch 5/30 ===== +


...


Metric Validation score improved (0.3042761305203805 --> 0.2996142655611038). Saving model!
train_loss 0.474488914818377 val_loss 0.2996142655611038 
val accuracy_score 0.8835443037974684 
 
+ ===== Epoch 27/30 ===== +



Metric Validation score improved (0.2996142655611038 --> 0.29839102694621455). Saving model!
train_loss 0.4298357927315944 val_loss 0.29839102694621455 
val accuracy_score 0.8911392405063291 
 
+ ===== Epoch 28/30 ===== +



EarlyStop count: 1 out of 5
train_loss 0.44733178494749837 val_loss 0.2988609763292166 
val accuracy_score 0.8911392405063291 
 
+ ===== Epoch 29/30 ===== +



Metric Validation score improved (0.29839102694621455 --> 0.2892174869775772). Saving model!
train_loss 0.4236105479098655 val_loss 0.2892174869775772 
val accuracy_score 0.8987341772151899 
 
+ ===== Epoch 30/30 ===== +



EarlyStop count: 1 out of 5
train_loss 0.44597222676148285 val_loss 0.2934233809892948 
val accuracy_score 0.9012658227848102 
 


In [ ]:
torch.save(model.state_dict(),"/content/drive/MyDrive/ACMCancer/last.pth")